In [ ]:
# Import google drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import shutil

In [ ]:
#Step 1: Preparing Dataset
images_dir = '/content/drive/My Drive/images'
if not os.path.exists(images_dir):
    os.makedirs(images_dir)


In [ ]:
# Creating training, validation and test directories
base_dir = '/content/drive/My Drive/splits'
os.makedirs(base_dir, exist_ok=True)

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Creating subdirectories for each category
for split_dir in [train_dir, val_dir, test_dir]:
    for category in ['cars', 'bicycles', 'mountains', 'deer']:
        os.makedirs(os.path.join(split_dir, category), exist_ok=True)

In [ ]:
# Spliting dataset into training, validation and test - 70%, 20% and 10% respectively
for category in ['cars', 'bicycles', 'mountains', 'deer']:
    category_dir = os.path.join(images_dir, category)

    if not os.path.exists(category_dir):
        print(f"Warning: Category directory not found: {category_dir}")
        continue

    images = os.listdir(category_dir)

    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.3, random_state=42)

    # Moving images to the directories where they belong
    for image in train_images:
        source_path = os.path.join(category_dir, image)
        destination_path = os.path.join(train_dir, category, image)

        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)
        else:
            print(f"Warning: Image file not found: {source_path}")

    for image in val_images:
        shutil.copy(os.path.join(category_dir, image), os.path.join(val_dir, category, image))
    for image in test_images:
        shutil.copy(os.path.join(category_dir, image), os.path.join(test_dir, category, image))

print("Dataset successfully split into training, validation, and test sets!")

Dataset successfully split into training, validation, and test sets!


In [ ]:
# Defining the function that deletes corrupt images
from PIL import Image
import os

def delete_corrupt_images(directory):
    for subdir, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(subdir, file)
            try:
                with Image.open(file_path) as img:
                    img.convert("RGB").save(file_path)
            except (OSError, IOError):
                print(f"Deleting corrupt image: {file_path}")
                os.remove(file_path)

# Apply to all dataset folders
for folder in [train_dir, val_dir, test_dir]:
    delete_corrupt_images(folder)

print("All corrupt images removed successfully!")

Deleting corrupt image: /content/drive/My Drive/splits/train/deer/Nagano's_deer_(49785030382).jpg
Deleting corrupt image: /content/drive/My Drive/splits/train/deer/.DS_Store
All corrupt images removed successfully!


In [ ]:
# Step 2: Data Augmentation and Generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 1065 images belonging to 4 classes.
Found 304 images belonging to 4 classes.
Found 154 images belonging to 4 classes.


In [ ]:
# Step 3: Model Design
# Using a CNN (Convolutional Neural Network) because of its effectiveness. Less parameters required, pooling layers and handles colour channels and depth well.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

# Compiling the Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Step 4: Model Training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    batch_size=32
)

Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 315s 9s/step - accuracy: 0.7246 - loss: 0.7144 - val_accuracy: 0.7039 - val_loss: 0.8542
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 296s 9s/step - accuracy: 0.6946 - loss: 0.7987 - val_accuracy: 0.7467 - val_loss: 0.7337
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 290s 8s/step - accuracy: 0.7075 - loss: 0.7433 - val_accuracy: 0.7138 - val_loss: 0.8099
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 293s 9s/step - accuracy: 0.7028 - loss: 0.7656 - val_accuracy: 0.6480 - val_loss: 0.8935
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 301s 9s/step - accuracy: 0.7268 - loss: 0.7542 - val_accuracy: 0.6579 - val_loss: 0.9453
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 309s 9s/step - accuracy: 0.7097 - loss: 0.7464 - val_accuracy: 0.7105 - val_loss: 0.8168
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 291s 9s/step - accuracy: 0.7052 - loss: 0.7415 - val_accuracy: 0.7664 - val_loss: 0.6909
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 291s 9s/step - accuracy: 0.7277 - loss: 0.6982 - val_accuracy: 0.6711 - v

In [ ]:
# Step 5: Model Evaluation
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 25s 5s/step - accuracy: 0.7416 - loss: 0.7652
Test Accuracy: 73.38%


In [ ]:
# Step 6: Save the Model
model.save('image_classifier.h5')

In [ ]:
# Step 7: Define the function to classify new images
def classify_new_image(img_path, tf_model, class_names):

    # Loading and preprocessing the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    # Predicting the class
    predictions = tf_model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]

    # Displaying the image and prediction
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_class_name}")
    plt.show()

    return predicted_class_name


In [ ]:
# Loading the saved model
loaded_model = tf.keras.models.load_model('image_classifier.h5')

In [ ]:
# Define model path in Google Drive
drive_model_path = '/content/drive/My Drive/image_classifier.h5'

# Save the model
loaded_model.save('image_classifier.h5')

# Copy model to Google Drive
shutil.copy('image_classifier.h5', drive_model_path)

print(f"Model saved to {drive_model_path}")

Model saved to /content/drive/My Drive/image_classifier.h5
